In [2]:
import keras
keras.backend.backend()

Using Theano backend.


'theano'

In [4]:
from sklearn import preprocessing
import numpy as np
from readers.data_reader import Dataset
from experiments.emotion_flow_exp import ExperimentSentimentAttentionMT
import sys
from sklearn.metrics import f1_score

In [15]:
model_root='../models/'

model='sf_mt_50_checked_model.hdf5'

param={'recurrent_units': 64, 'recurrent_dropout': 0.3, 'recurrent_activation': 'tanh', 'patience': 25, 
       'lr': 0.001, 'initializer': 'lecun_uniform', 'genre_emb_size': 32, 'epochs': 100, 'dropout_1': 0.2,
       'dropout': 0.2, 'batch_size': 1, 'attention_units': 64}

In [6]:
feature_lst=['nrc_mean_sd_50_all'] 

dataset = Dataset(features=feature_lst, label_extractor='success_genre', genre=False)

In [7]:
X_train, Y_train = dataset.get_training_data()
X_val, Y_val = dataset.get_validaiontion_data()
X_test, Y_test = dataset.get_test_data()

le = preprocessing.LabelEncoder()

Y_train['success_output'] = le.fit_transform(Y_train['success_output'])
Y_val['success_output'] = le.transform(Y_val['success_output'])
Y_test['success_output'] = le.transform(Y_test['success_output'])

Y_test_books=dataset.info['test_books']
Y_test_Genre= dataset.info['Y_test'][:,1]

In [8]:
for feature in set(feature_lst) :
    print("Feature: {}".format(feature))
    print("Traning shape {}:{}".format(feature, X_train[feature].shape))
    print("Val shape {}:{}".format(feature, X_val[feature].shape))
    print("Test shape {}:{}".format(feature, X_test[feature].shape))
    print()

Feature: nrc_mean_sd_50_all
Traning shape nrc_mean_sd_50_all:(555, 50, 20)
Val shape nrc_mean_sd_50_all:(139, 50, 20)
Test shape nrc_mean_sd_50_all:(290, 50, 20)



In [9]:
Y_test_books[:5]

['24875_lucy+maud+montgomery+short+stories+1904.txt',
 '26060_complete+version+of+ye+three+blind+mice.txt',
 '23162_no+great+magic.txt',
 '1257_the+three+musketeers.txt',
 '2059_the+little+shepherd+of+kingdom+come.txt']

In [10]:
Y_test_Genre[:5]

array(['Short_stories', 'Poetry', 'Science_fiction', 'Historical_fiction',
       'Historical_fiction'], dtype=object)

###  Load model and predict

In [11]:
experiment = ExperimentSentimentAttentionMT(train_Xs=X_train, train_Ys=Y_train, val_Xs=X_val, val_Ys=Y_val,
                                          test_Xs=X_test,
                                          test_Ys=Y_test, le=le,
                                          params=param,
                                          name='att_MT_')

In [16]:
exp_model_path=model_root+model
experiment.predict(checked_model_path=exp_model_path,settings=param)

/anaconda3/lib/python3.6/site-packages/keras/layers/recurrent.py:1550: UserWarning: RNN dropout is no longer supported with the Theano backend due to technical limitations. You can either set `dropout` and `recurrent_dropout` to 0, or use the TensorFlow backend.
  'RNN dropout is no longer supported with the Theano backend '


MT prediction task
F1 Weighted Score: 0.6896466217917353


In [17]:
from keras.models import load_model

model=experiment.create_model(param)
model.load_weights(exp_model_path)
Y_test_pred = model.predict(X_test)
if len(Y_test_pred) == 2:
    print("MT prediction task")
    Y_test_pred = Y_test_pred[0].flatten()
Y_test_pred = np.array([1 if y > 0.5 else 0 for y in Y_test_pred])

print ("F1 Weighted Score: {}".format(f1_score(Y_test['success_output'].flatten(),
                                          Y_test_pred.flatten(), average='weighted', pos_label=None)))

/anaconda3/lib/python3.6/site-packages/keras/layers/recurrent.py:1550: UserWarning: RNN dropout is no longer supported with the Theano backend due to technical limitations. You can either set `dropout` and `recurrent_dropout` to 0, or use the TensorFlow backend.
  'RNN dropout is no longer supported with the Theano backend '


MT prediction task
F1 Weighted Score: 0.6896466217917353
